In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1=pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/o2Saturation.csv')
df1.head()
#df1.shape

#here there are vey few rows comparing to other dataset,so for now going to ignore other datasset


## INFORMATION OF CLMS IN DATASETS
Age : Age of the patient

Sex : Sex of the patient

exang: exercise induced angina (1 = yes; 0 = no)

ca: number of major vessels (0-3)

cp : Chest Pain type chest pain type

Value 1: typical angina
Value 2: atypical angina
Value 3: non-anginal pain
Value 4: asymptomatic
trtbps : resting blood pressure (in mm Hg)

chol : cholestoral in mg/dl fetched via BMI sensor

fbs : (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)

rest_ecg : resting electrocardiographic results

Value 0: normal
Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
thalach : maximum heart rate achieved

target : 0= less chance of heart attack 1= more chance of heart attack

In [ ]:
df=pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
df.head()
#here it seems from coomunity discusstion and outputs that the target columns has been changeror tempered with

In [ ]:
df.info()
#here we can see the null values present in the data set
#there are no null values
#all the columns are in int or float format though ,we can see that sex colomn is not described o,1

In [ ]:
#we are dropin cp,fbs ,sex and output coloumn dor describe function 
df.drop(['sex','output','cp','fbs'],axis=1).describe()
#here you can see that many columns contain 0,1,2, values 
#for better understanding of the data we can drop this from the describe to


# EXPLORATORY DATA ANALYSIS


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:

fig_dims = (15,8)
fig, ax = plt.subplots(figsize=fig_dims)
#sns.barplot(x = "names", y = "values", ax=ax, data=df)
sns.countplot(x=df.age,hue=df.output,ax=ax,)
#0=less chance of heart attack, 1=Higher chance

In [ ]:
plt.hist(x=df['cp'])
plt.xlabel('Cp')
#here CP 0,1,2,3 repressent differnt types od agnima

In [ ]:
#we can see the relationship between output and sex 

fig_dims = (6,6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x=df.sex,hue=df.output,ax=ax,)
#here in the dataset we are not sure about which is male and female in 0,1 because of lack of information

In [ ]:
df.head()

In [ ]:
df_cp=df[['cp','exng','output']].copy()

In [ ]:
#now we will see the relation between cp(Chest pain) exng(excercise induced pain) and output
#Value 1: typical angina Value 2: atypical angina Value 3: non-anginal pain Value 4: asymptomatic trtbps : resting blood pressure (in mm Hg)
df_c = df[['cp', 'exng', 'output']].copy()
df_c['cp'] = df_c['cp'].map({0:'typical angina', 1:'atypical angina', 2:'non-anginal pain', 3:'asymptomatic'})
df_c['exng'] = df_c['exng'].map({0:'No', 1:'Yes'})
df_c.columns = ['Type of Chest Pain', 'Exercise Induced Angina', 'Chance of Heart Attack']
df_c['Chance of Heart Attack'] = df_c['Chance of Heart Attack'].map({0:'Lower', 1:'Higher'})
df_c.head()

In [ ]:
#realoising the relation betwwen chestpain and inuced agnima
fig_dims = (8,8)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(x=df_c['Type of Chest Pain'],hue=df_c['Exercise Induced Angina'],ax=ax,)
#one can see that there is a strong relation between typicalangina and exercise induced angina

In [ ]:
sns.countplot(x=df_c['Exercise Induced Angina'],hue=df_c['Chance of Heart Attack'])
#here we can see that where angina is not caused by exercise than the chamce of attack is higher
#which is quiety unsual

In [ ]:
df.corr()

In [ ]:
fig_dims = (10,10)
fig, ax = plt.subplots(figsize=fig_dims)
sns.color_palette("flare", as_cmap=True)
sns.heatmap(df.corr(),annot=True)
#here we ploted the corelation plot for the better understanding of which feature is connected to output or target
#if mlore than one feature has higher co relation between than you can neglect that parameter for model
# we can see that thalachh as hiher relation foroutput lets see

In [ ]:
#now we will se the relations between some other features and attribute ,lets see columns first
df.columns

In [ ]:
#we can chek the relationship beetween trtbps and chol
df['trtbps'].head()
df['chol'].head()
df_a=df[['trtbps','chol','output','thall']].copy()

In [ ]:
df_a.corr()
sns.jointplot(x = df_a['trtbps'], y = df_a['chol'],
              kind = "scatter", data = df_a,hue='output')
#sns.regplot(x=df_a['trtbps'],y = df_a['chol'])
#0-lower chance 1-higher chance

In [ ]:
df_high=df[df['output']==1][['chol','output']]
df_high.head()
#now we can use describe function on chol to get more insights
df_high.describe()
#here we can see the min max and mean cholestrol levelfor higher chance of having a heart attacck


# Building Differnt ML models for given Problem

In [ ]:
#Building data for ML models
from sklearn.model_selection import train_test_split

X=df.drop('output',axis=1).values
#type(X)
#X.shape
y=df['output'].values

In [ ]:
# Split it to a 70:30 Ratio Train:Test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# Normalize the data
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#this only to see how the data set lokks like after normalization
df_train = pd.DataFrame(X_train)
df_train.head()

****Using Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)

predictions = model.predict(X_test)

print("Accuracy {0:.2f}%".format(100*accuracy_score(predictions, y_test)))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
cm_lr=confusion_matrix(y_test, predictions)


In [ ]:
sns.heatmap(data=cm_lr,annot=True)
# 0 is less chance of heart attack and 1 is higher chance of heart attack 
#this is visual representation of confusion matrix

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

model = RandomForestClassifier()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
score = model.score(X_test, y_test)

print("Accuracy {0:.2f}%".format(100*accuracy_score(predictions, y_test)))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
cm_rf=confusion_matrix(y_test, predictions)

In [ ]:
sns.heatmap(data=cm_rf,annot=True)
# 0 is less chance of heart attack and 1 is higher chance of heart attack 
#this is visual representation of confusion matrix

In [ ]:
X_test.shape

In [ ]:
#%tensorflow_version 2.x 
# If you wish to use Tensorflow 1.X run the following line and then restart runtime
# %tensorflow_version 1.x 
# You'll need to change your import statements from tensorflow.keras to keras
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=13))
model.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))

model.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

In [ ]:
model.summary()

In [ ]:
batch_size = 15
epochs = 40
#change epoch and batch size for different results


history = model.fit(X_train,
                    y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
predictions = model.predict(X_test)
predictions = (predictions > 0.5)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

Above all the results are based pn different training data,for number of different iterations Neural network and random forest classifiers looks fine with accuracy between 80 -85
hopefully after fyrther evalution i can improve this  models tuning